## Import dependencies

In [1]:
import numpy as np
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import scipy as sp
from tqdm import tqdm
import glob

from fair import *
from fair.scripts.data_retrieval import *

%matplotlib inline

# RCMIP simulations
In this notebook we run the simulations to be submitted to RCMIP Phase II. These are simulations over various key experiments using a set of 5000 parameters from the CONSTRAINED parameter ensemble.

**This notebook is not well commented as it does not form part of the FaIRv2.0.0 paper.**

In [2]:
## first, import the categories for aggregation & adjust to match RCMIP:
param_categories = pd.read_csv('../../aux/FaIRv2.0.0-alpha_RF_categories.csv',index_col=0,skiprows=1,names=['source','category'])
f_gas_list = ['c2f6', 'c3f8', 'c4f10', 'c5f12', 'c6f14', 'c7f16', 'c8f18', 'c_c4f8','nf3', 'sf6', 'so2f2','cf4','hfc125', 'hfc134a', 'hfc143a', 'hfc152a', 'hfc227ea', 'hfc236fa', 'hfc23', 'hfc245fa', 'hfc32', 'hfc365mfc', 'hfc4310mee']
montreal_gas_list = ['carbon_tetrachloride','cfc113', 'cfc114', 'cfc115', 'cfc11', 'cfc12', 'ch2cl2', 'ch3ccl3', 'chcl3', 'halon1211', 'halon1301', 'halon2402', 'hcfc141b', 'hcfc142b', 'hcfc22','halon1202','methyl_bromide', 'methyl_chloride']
param_categories.loc[f_gas_list] = 'f_gases'
param_categories.loc[montreal_gas_list] = 'montreal_gases'
param_categories.loc['Total'] = 'Total'

RCMIP_outputmap = pd.read_csv('../../aux/FaIRv2.0.0-alpha_RCMIP_inputmap.csv',index_col=0)
RCMIP_forcmap = pd.read_csv('../../aux/FaIRv2.0.0-alpha_RCMIP_forcmap.csv',index_col=0)
RCMIP_forcmap.loc['ozone'] = 'Effective Radiative Forcing|Anthropogenic|Other|Ozone'

Variables required as per the paper draft + SI:
- GSAT / GMST
- Total ERF
- Aerosol ERF
- CO2 concs (esm)
- CO2 ERF
- CH4 ERF
- N2O ERF
- F-gas ERF
- OHC (not available)
- Trop O3

### obtain a set of 5000 CONSTRAINED parameters

In [3]:
## get a set of ensemble selection likelihoods
datasets_to_use = ['HadCRUT5']#['HadCRUT4','HadCRUT4','NOAA','GISTEMP','CW','BERKELEY']

FULL_probabilities = pd.concat([pd.read_hdf('../../aux/parameter-sets/perturbed-parameters/FULL_selection_probability-'+x+'.h5') for x in datasets_to_use],axis=1,keys=datasets_to_use)
FULL_ensemble_selection = FULL_probabilities.mean(axis=1)>np.random.random(FULL_probabilities.shape[0])

In [4]:
N_sets = 5000

# here we randomly select which ensemble members will make up our parameter sets
CONSTRAINED_ensemble_members = FULL_ensemble_selection.index[FULL_ensemble_selection][np.random.choice(FULL_ensemble_selection.sum(),N_sets,replace=False)]

In [5]:
ALL_mems = [x.split('/')[-1].split('.')[0] for x in glob.glob('../../aux/parameter-sets/perturbed-parameters/gas_params/*.h5')]

CONSTRAINED_thermal_set = []
CONSTRAINED_gas_set = []
CONSTRAINED_extforc_sfs = []

## and here we generate the parameter sets
for mem_range in ALL_mems:
    
    gas_params = pd.read_hdf('../../aux/parameter-sets/perturbed-parameters/gas_params/'+mem_range+'.h5')
    thermal_params = pd.read_hdf('../../aux/parameter-sets/perturbed-parameters/climresp_params/FULL/'+mem_range+'.h5')
    extforc_sfs = pd.read_hdf('../../aux/parameter-sets/perturbed-parameters/extforc_sfs/'+mem_range+'.h5')
    
    CONSTRAINED_mems = set(gas_params.columns.levels[0]).intersection(CONSTRAINED_ensemble_members)
    
    CONSTRAINED_thermal_set += [thermal_params.reindex(CONSTRAINED_mems,axis=1,level=0)]
    CONSTRAINED_gas_set += [gas_params.reindex(CONSTRAINED_mems,axis=1,level=0)]
    CONSTRAINED_extforc_sfs += [extforc_sfs.reindex(CONSTRAINED_mems,axis=1)]
    
CONSTRAINED_thermal_set = pd.concat(CONSTRAINED_thermal_set,axis=1)
CONSTRAINED_gas_set = pd.concat(CONSTRAINED_gas_set,axis=1)
CONSTRAINED_extforc_sfs = pd.concat(CONSTRAINED_extforc_sfs,axis=1)

### Create FaIRv2.0.0-alpha metadata

In [6]:
metadata = pd.DataFrame(columns = ['climate_model','climate_model_name','climate_model_version','climate_model_configuration_label','climate_model_configuration_description','project','name_of_person','literature_reference'],index=['entry'])
metadata.loc['entry','climate_model'] = 'FaIRv2.0.0-alpha'
metadata.loc['entry','climate_model_name'] = 'FaIR'
metadata.loc['entry','climate_model_version'] = '2.0.0'
metadata.loc['entry','climate_model_configuration_label'] = 'CONSTRAINED-HadCRUT5'
metadata.loc['entry','climate_model_configuration_description'] = 'FaIRv2.0.0-alpha large parameter ensemble subset with HadCRUT5-based AWI constraint applied.'
metadata.loc['entry','project'] = 'RCMIP'
metadata.loc['entry','name_of_person'] = 'Nicholas Leach <nicholas.leach@stx.ox.ac.uk>'
metadata.loc['entry','literature_reference'] = 'https://doi.org/10.5194/gmd-2020-390'

metadata.to_csv('../../aux/output-data/RCMIP/meta_FaIRv2.0.0-alpha-HC5.csv',index=False)

### Create dataframe for key ensemble member metrics

In [7]:
member_metrics = pd.read_hdf('../../aux/parameter-sets/perturbed-parameters/FULL_ALL.h5')

member_metrics = member_metrics[['ECS','TCR','F2xCO2']].reindex(CONSTRAINED_ensemble_members)
member_metrics = member_metrics.unstack().rename({'ECS':'Equilibrium Climate Sensitivity','TCR':'Transient Climate Response','F2xCO2':'F-2xCO2'},level=0).reset_index().rename({'level_0':'RCMIP name','level_1':'ensemble_member',0:'value'},axis=1)
member_metrics.loc[:,'unit'] = 'K'
member_metrics.loc[:,'climate_model'] = 'FaIRv2.0.0-alpha'
member_metrics.loc[member_metrics['RCMIP name']=='F-2xCO2','unit'] = 'W/m^2'
member_metrics.loc[:,'ensemble_member'] = member_metrics.loc[:,'ensemble_member'].str[3:].astype(int)

member_metrics.loc[member_metrics['RCMIP name']=='Equilibrium Climate Sensitivity'].to_csv('../../aux/output-data/RCMIP/reported-metrics_FaIRv2.0.0-alpha-HC5.csv.gz', compression='gzip',index=False)

## Concentration-driven experiments

### 1pctCO2 , abrupt2xCO2, abrupt4xCO2, abrupt0.5xCO2

In [8]:
## create the scenarios:

idealised_concs = pd.DataFrame(284*1.01**(np.arange(201)),index=np.arange(201),columns=pd.MultiIndex.from_product([['1pctCO2'],['carbon_dioxide']]))
idealised_concs.loc[:,('abrupt-4xCO2','carbon_dioxide')] = 284*4
idealised_concs.loc[:,('abrupt-2xCO2','carbon_dioxide')] = 284*2
idealised_concs.loc[:,('abrupt-0p5xCO2','carbon_dioxide')] = 284/2
idealised_concs.loc[0,(slice(None),'carbon_dioxide')] = 284

idealised_forc = return_empty_forcing(idealised_concs)

idealised_params = CONSTRAINED_gas_set.reindex(['carbon_dioxide'],axis=1,level=1)
idealised_params.loc['PI_conc']=284

In [9]:
## run the experiments
idealised_exp = run_FaIR(concentrations_in=idealised_concs,forcing_in=idealised_forc,gas_parameters=idealised_params,thermal_parameters=CONSTRAINED_thermal_set)

Integrating 4 scenarios, 5000 gas cycle parameter sets, 1 thermal response parameter sets, over ['carbon_dioxide'] forcing agents, between 0 and 200...


100%|██████████| 200/200 [00:02<00:00, 89.11 timestep/s]


In [10]:
## temperature output
T_out = idealised_exp['T'].loc[1:200].copy()
T_out.index = np.arange(200)

T_out = T_out.T.reset_index().rename({'Gas cycle set':'ensemble_member','Scenario':'scenario'},axis=1)

T_out.loc[:,'variable'] = 'Surface Air Temperature Change'
T_out.loc[:,'region'] = 'World'
T_out.loc[:,'climate_model'] = 'FaIRv2.0.0-alpha'
T_out.loc[:,'model'] = 'idealised'
T_out.loc[:,'unit'] = 'K'

T_out = T_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()

In [11]:
## RF output
RF_out = idealised_exp['RF'].loc[1:200].xs('carbon_dioxide',axis=1,level=-1).copy()
RF_out.index = np.arange(200)

RF_out = RF_out.T.reset_index().rename({'Gas cycle set':'ensemble_member','Scenario':'scenario'},axis=1)

RF_out.loc[:,'variable'] = 'Effective Radiative Forcing'
RF_out.loc[:,'region'] = 'World'
RF_out.loc[:,'climate_model'] = 'FaIRv2.0.0-alpha'
RF_out.loc[:,'model'] = 'idealised'
RF_out.loc[:,'unit'] = 'W/m^2'

RF_out = RF_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()

In [12]:
## Emissions out
emms_out = idealised_exp['Emissions'].loc[1:200].droplevel(axis=1,level=-1)/RCMIP_outputmap.loc['carbon_dioxide','RCMIP_emms_scaling']
emms_out.index = np.arange(200)

emms_out = emms_out.T.reset_index().rename({'Gas cycle set':'ensemble_member','Scenario':'scenario'},axis=1)

emms_out.loc[:,'variable'] = 'Emissions|CO2'
emms_out.loc[:,'region'] = 'World'
emms_out.loc[:,'climate_model'] = 'FaIRv2.0.0-alpha'
emms_out.loc[:,'model'] = 'idealised'
emms_out.loc[:,'unit'] = 'Mt CO2/yr'

emms_out = emms_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()

In [13]:
## Remaining variables
atmos_pool_out = emms_out.loc[emms_out.variable=='Emissions|CO2'].copy()

for experiment in atmos_pool_out.scenario.unique():
    atmos_pool_out.loc[atmos_pool_out.scenario==experiment,0:] = ((idealised_exp['C'].loc[1:,(experiment,'carbon_dioxide')]-284)/(4.688876e-01*RCMIP_outputmap.loc['carbon_dioxide','RCMIP_emms_scaling'])).values

atmos_pool_out.loc[:,'variable'] = 'Carbon Pool|Atmosphere'
atmos_pool_out.loc[:,'unit'] = 'Mt CO2'

In [14]:
output = pd.concat([T_out,RF_out,emms_out,atmos_pool_out],axis=0)
output.iloc[:,7:] = output.iloc[:,7:].astype(np.single)
output.columns = output.columns.tolist()[:7]+list(range(1850,2050))
output.loc[:,'ensemble_member'] = output.loc[:,'ensemble_member'].str[3:].astype(int)

output.loc[:,:1999].to_csv('../../aux/output-data/RCMIP/idealised_FaIRv2.0.0-alpha-HC5.csv.gz', compression='gzip',index=False)

### SSPs

In [15]:
def run_RCMIP_ssp(ssp):

    ## retrieve the data for the ssp
    concs = RCMIP_to_FaIR_input_concs(ssp).loc[1750:2300]
    emms = RCMIP_to_FaIR_input_emms(ssp).interpolate().loc[1750:2300]

    aer_species = ['so2','nox','co','nmvoc','bc','nh3','oc','nox_avi']
    concs.loc[:,aer_species] = emms.loc[:,aer_species] - emms.loc[1750,aer_species]

    # No SSP data for Halon 1202 so set to zero
    concs['halon1202'] = 0

    LUC_forc = pd.concat([get_RCMIP_forc([ssp],'Effective Radiative Forcing|Anthropogenic|Albedo Change')]*N_sets,axis=1,keys=CONSTRAINED_extforc_sfs.loc['LUC'].index)*CONSTRAINED_extforc_sfs.loc['LUC'].values

    nat_forc = get_RCMIP_forc([ssp],['Effective Radiative Forcing|Natural'])

    forc = (LUC_forc + nat_forc.values).loc[concs.index]

    concs = pd.concat([concs]*N_sets,axis=1,keys=forc.columns.levels[0])

    ## run the model:
    ### NB. No Halon1202 data for the SSPs so remove from the inputs
    result = run_FaIR(concentrations_in=concs,
                      forcing_in=forc,
                      gas_parameters=CONSTRAINED_gas_set,
                      thermal_parameters=CONSTRAINED_thermal_set,
                      aer_concs_in=aer_species,
                      show_run_info=False)

    ## temperature output
    T_out = pd.concat([result['T'].loc[:].copy()],axis=1,keys=[ssp],names=['Scenario','Gas cycle set'])
    T_out = T_out.T.reset_index().rename({'Gas cycle set':'ensemble_member','Scenario':'scenario'},axis=1)
    T_out.loc[:,'variable'] = 'Surface Air Temperature Change'
    T_out.loc[:,'region'] = 'World'
    T_out.loc[:,'climate_model'] = 'FaIRv2.0.0-alpha'
    T_out.loc[:,'model'] = RCMIP_concs.loc[('World',ssp)].Model.unique()[0]
    T_out.loc[:,'unit'] = 'K'
    T_out = T_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()

    ## RF output - excluding ozone, strath20, bconsnow, contrails
    RF_out = result['RF'].stack(level=0).groupby(param_categories.category.to_dict(),axis=1).sum().drop(['ozone','strat_h2o','contrails','bc_on_snow'],axis=1).rename(RCMIP_forcmap.RCMIP_forc_key.to_dict(),axis=1).stack().unstack(level=0).reset_index()

    RF_out = RF_out.rename({'Scenario':'ensemble_member','level_1':'variable'},axis=1)
    RF_out.loc[:,'scenario'] = ssp
    RF_out.loc[:,'region'] = 'World'
    RF_out.loc[:,'climate_model'] = 'FaIRv2.0.0-alpha'
    RF_out.loc[:,'model'] = RCMIP_concs.loc[('World',ssp)].Model.unique()[0]
    RF_out.loc[:,'unit'] = 'W/m^2'

    RF_out = RF_out.loc[:,['climate_model','model','scenario','region','variable','unit','ensemble_member']+list(range(1750,2301))]

    ## Emissions out
    emms_out = result['Emissions'].xs('carbon_dioxide',axis=1,level=-1)/RCMIP_outputmap.loc['carbon_dioxide','RCMIP_emms_scaling']

    emms_out = emms_out.T.reset_index().rename({'Scenario':'ensemble_member'},axis=1)

    emms_out.loc[:,'variable'] = 'Emissions|CO2'
    emms_out.loc[:,'scenario'] = ssp
    emms_out.loc[:,'region'] = 'World'
    emms_out.loc[:,'climate_model'] = 'FaIRv2.0.0-alpha'
    emms_out.loc[:,'model'] = RCMIP_concs.loc[('World',ssp)].Model.unique()[0]
    emms_out.loc[:,'unit'] = 'Mt CO2/yr'

    emms_out = emms_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()

    ## Remaining variable(s)
    atmos_pool_out = emms_out.loc[emms_out.variable=='Emissions|CO2'].copy()

    atmos_pool_out.loc[:,1750:] = ((result['C'].xs('carbon_dioxide',axis=1,level=-1)-278)/(4.688876e-01*RCMIP_outputmap.loc['carbon_dioxide','RCMIP_emms_scaling'])).values.T

    atmos_pool_out.loc[:,'variable'] = 'Carbon Pool|Atmosphere'
    atmos_pool_out.loc[:,'unit'] = 'Mt CO2'
    
    output = pd.concat([T_out,RF_out,emms_out,atmos_pool_out])
    output.iloc[:,7:] = output.iloc[:,7:].astype(np.single)
    output.loc[:,'ensemble_member'] = output.loc[:,'ensemble_member'].str[3:].astype(int)

    print('saving',ssp)
    output.to_csv('../../aux/output-data/RCMIP/'+ssp+'_FaIRv2.0.0-alpha-HC5.csv.gz', compression='gzip',index=False)

In [16]:
for ssp in ['ssp119','ssp126','ssp245','ssp370','ssp370-lowNTCF-aerchemmip','ssp370-lowNTCF-gidden','ssp434','ssp460','ssp534-over','ssp585']:
    run_RCMIP_ssp(ssp)

100%|██████████| 550/550 [01:34<00:00,  5.80 timestep/s]
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:36: PerformanceWarning: indexing past lexsort depth may impact performance.
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:47: PerformanceWarning: indexing past lexsort depth may impact performance.
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:61: PerformanceWarning: indexing past lexsort depth may impact performance.


saving ssp119


100%|██████████| 550/550 [01:36<00:00,  5.68 timestep/s]
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:36: PerformanceWarning: indexing past lexsort depth may impact performance.
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:47: PerformanceWarning: indexing past lexsort depth may impact performance.
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:61: PerformanceWarning: indexing past lexsort depth may impact performance.


saving ssp126


100%|██████████| 550/550 [01:52<00:00,  4.88 timestep/s]
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:36: PerformanceWarning: indexing past lexsort depth may impact performance.
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:47: PerformanceWarning: indexing past lexsort depth may impact performance.
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:61: PerformanceWarning: indexing past lexsort depth may impact performance.


...

saving ssp460


100%|██████████| 550/550 [01:41<00:00,  5.40 timestep/s]
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:36: PerformanceWarning: indexing past lexsort depth may impact performance.
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:47: PerformanceWarning: indexing past lexsort depth may impact performance.
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:61: PerformanceWarning: indexing past lexsort depth may impact performance.


saving ssp534-over


100%|██████████| 550/550 [01:34<00:00,  5.81 timestep/s]
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:36: PerformanceWarning: indexing past lexsort depth may impact performance.
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:47: PerformanceWarning: indexing past lexsort depth may impact performance.
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:61: PerformanceWarning: indexing past lexsort depth may impact performance.


saving ssp585


### RCPs
TODO

## Emission-driven experiments

### esm-SSP
We run the esm-SSP simulations using CO2 emissions plus total forcing (excluding CO2) from the concentration-driven simulations.

In [17]:
def run_RCMIP_esm_ssp(ssp):

    ## retrieve the data for the ssp
    concs = RCMIP_to_FaIR_input_concs(ssp).loc[1750:2300]
    emms = RCMIP_to_FaIR_input_emms(ssp).interpolate().loc[1750:2300]

    aer_species = ['so2','nox','co','nmvoc','bc','nh3','oc','nox_avi']
    concs.loc[:,aer_species] = emms.loc[:,aer_species] - emms.loc[1750,aer_species]

    # No SSP data for Halon 1202 so set to zero
    concs['halon1202'] = 0

    LUC_forc = pd.concat([get_RCMIP_forc([ssp],'Effective Radiative Forcing|Anthropogenic|Albedo Change')]*N_sets,axis=1,keys=CONSTRAINED_extforc_sfs.loc['LUC'].index)*CONSTRAINED_extforc_sfs.loc['LUC'].values

    nat_forc = get_RCMIP_forc([ssp],['Effective Radiative Forcing|Natural'])

    forc = (LUC_forc + nat_forc.values).loc[concs.index]

    concs = pd.concat([concs]*N_sets,axis=1,keys=forc.columns.levels[0])

    concrun_rf = run_FaIR(concentrations_in=concs,
                          forcing_in=forc,
                          gas_parameters=CONSTRAINED_gas_set,
                          thermal_parameters=CONSTRAINED_thermal_set,
                          aer_concs_in=aer_species,
                          show_run_info=False)['RF']

    presc_forc = pd.concat([concrun_rf.xs('Total',axis=1,level=1)-concrun_rf.xs('carbon_dioxide',axis=1,level=1)],axis=1,keys=['forcing']).swaplevel(0,1,axis=1)
    emms = pd.concat([emms[['carbon_dioxide']]]*N_sets,axis=1,keys=presc_forc.columns.levels[0])

    result = run_FaIR(emissions_in=emms,
                      forcing_in=presc_forc,
                      gas_parameters=CONSTRAINED_gas_set.reindex(['carbon_dioxide'],axis=1,level=1),
                      thermal_parameters=CONSTRAINED_thermal_set,
                      show_run_info=False)

    ## temperature output
    T_out = pd.concat([result['T'].loc[:].copy()],axis=1,keys=[ssp],names=['Scenario','Gas cycle set'])
    T_out = T_out.T.reset_index().rename({'Gas cycle set':'ensemble_member','Scenario':'scenario'},axis=1)
    T_out.loc[:,'variable'] = 'Surface Air Temperature Change'
    T_out.loc[:,'region'] = 'World'
    T_out.loc[:,'climate_model'] = 'FaIRv2.0.0-alpha'
    T_out.loc[:,'model'] = RCMIP_concs.loc[('World',ssp)].Model.unique()[0]
    T_out.loc[:,'unit'] = 'K'
    T_out = T_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()

    ## RF output
    RF_out = result['RF'].xs('carbon_dioxide',axis=1,level=-1)

    RF_out = RF_out.T.reset_index().rename({'Scenario':'ensemble_member'},axis=1)
    RF_out.loc[:,'variable'] = 'Effective Radiative Forcing|Anthropogenic|CO2'
    RF_out.loc[:,'scenario'] = ssp
    RF_out.loc[:,'region'] = 'World'
    RF_out.loc[:,'climate_model'] = 'FaIRv2.0.0-alpha'
    RF_out.loc[:,'model'] = RCMIP_concs.loc[('World',ssp)].Model.unique()[0]
    RF_out.loc[:,'unit'] = 'W/m^2'

    RF_out = RF_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()

    ## Concentrations out
    concs_out = result['C'].xs('carbon_dioxide',axis=1,level=-1)

    concs_out = concs_out.T.reset_index().rename({'Scenario':'ensemble_member'},axis=1)

    concs_out.loc[:,'variable'] = 'Atmospheric Concentrations|CO2'
    concs_out.loc[:,'scenario'] = ssp
    concs_out.loc[:,'region'] = 'World'
    concs_out.loc[:,'climate_model'] = 'FaIRv2.0.0-alpha'
    concs_out.loc[:,'model'] = RCMIP_concs.loc[('World',ssp)].Model.unique()[0]
    concs_out.loc[:,'unit'] = 'ppm'

    concs_out = concs_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()
    
    ## Emissions out
    emms_out = result['Emissions'].xs('carbon_dioxide',axis=1,level=-1)/RCMIP_outputmap.loc['carbon_dioxide','RCMIP_emms_scaling']

    emms_out = emms_out.T.reset_index().rename({'Scenario':'ensemble_member'},axis=1)

    emms_out.loc[:,'variable'] = 'Emissions|CO2'
    emms_out.loc[:,'scenario'] = ssp
    emms_out.loc[:,'region'] = 'World'
    emms_out.loc[:,'climate_model'] = 'FaIRv2.0.0-alpha'
    emms_out.loc[:,'model'] = RCMIP_concs.loc[('World',ssp)].Model.unique()[0]
    emms_out.loc[:,'unit'] = 'Mt CO2/yr'

    emms_out = emms_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()

    output = pd.concat([T_out,RF_out,concs_out,emms_out])
    output.iloc[:,7:] = output.iloc[:,7:].astype(np.single)
    output.loc[:,'ensemble_member'] = output.loc[:,'ensemble_member'].str[3:].astype(int)

    print('saving',ssp)
    output.to_csv('../../aux/output-data/RCMIP/esm-'+ssp+'_FaIRv2.0.0-alpha-HC5.csv.gz', compression='gzip',index=False)

In [18]:
for ssp in ['ssp119','ssp126','ssp245','ssp370','ssp370-lowNTCF-aerchemmip','ssp370-lowNTCF-gidden','ssp434','ssp460','ssp534-over','ssp585']:
    run_RCMIP_esm_ssp(ssp)

100%|██████████| 550/550 [00:01<00:00, 503.20 timestep/s]
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:43: PerformanceWarning: indexing past lexsort depth may impact performance.
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:55: PerformanceWarning: indexing past lexsort depth may impact performance.
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:69: PerformanceWarning: indexing past lexsort depth may impact performance.
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:83: PerformanceWarning: indexing past lexsort depth may impact performance.


saving ssp119


100%|██████████| 550/550 [00:00<00:00, 557.34 timestep/s]
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:43: PerformanceWarning: indexing past lexsort depth may impact performance.
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:55: PerformanceWarning: indexing past lexsort depth may impact performance.
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:69: PerformanceWarning: indexing past lexsort depth may impact performance.
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:83: PerformanceWarning: indexing past lexsort depth may impact performance.


saving ssp126


100%|██████████| 550/550 [00:01<00:00, 529.13 timestep/s]
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:43: PerformanceWarning: indexing past lexsort depth may impact performance.
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:55: PerformanceWarning: indexing past lexsort depth may impact performance.
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:69: PerformanceWarning: indexing past lexsort depth may impact performance.
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:83: PerformanceWarning: indexing past lexsort depth may impact performance.


...

saving ssp460


100%|██████████| 550/550 [00:01<00:00, 503.46 timestep/s]
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:43: PerformanceWarning: indexing past lexsort depth may impact performance.
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:55: PerformanceWarning: indexing past lexsort depth may impact performance.
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:69: PerformanceWarning: indexing past lexsort depth may impact performance.
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:83: PerformanceWarning: indexing past lexsort depth may impact performance.


saving ssp534-over


100%|██████████| 550/550 [00:00<00:00, 561.71 timestep/s]
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:43: PerformanceWarning: indexing past lexsort depth may impact performance.
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:55: PerformanceWarning: indexing past lexsort depth may impact performance.
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:69: PerformanceWarning: indexing past lexsort depth may impact performance.
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:83: PerformanceWarning: indexing past lexsort depth may impact performance.


saving ssp585


### esm-SSP-allGHG
Easier than esm-SSP in FaIRv2.0.0-alpha...

In [19]:
def run_RCMIP_esm_ssp_allGHG(ssp):

    ## retrieve the data for the ssp
    emms = RCMIP_to_FaIR_input_emms(ssp).interpolate().loc[1750:2300]
    ## rebase emission-driven forcings & species with natural emissions to 1750
    rebase_species = ['so2','nox','co','nmvoc','bc','nh3','oc','nox_avi','methyl_bromide','methyl_chloride','chcl3','ch2cl2']
    emms.loc[:,rebase_species] -= emms.loc[1750,rebase_species]

    LUC_forc = pd.concat([get_RCMIP_forc([ssp],'Effective Radiative Forcing|Anthropogenic|Albedo Change')]*N_sets,axis=1,keys=CONSTRAINED_extforc_sfs.loc['LUC'].index)*CONSTRAINED_extforc_sfs.loc['LUC'].values

    nat_forc = get_RCMIP_forc([ssp],['Effective Radiative Forcing|Natural'])

    forc = (LUC_forc + nat_forc.values).loc[emms.index]

    emms = pd.concat([emms]*N_sets,axis=1,keys=forc.columns.levels[0])

    result = run_FaIR(emissions_in=emms,
                      forcing_in=forc,
                      gas_parameters=CONSTRAINED_gas_set,
                      thermal_parameters=CONSTRAINED_thermal_set,
                      show_run_info=False)

    ## temperature output
    T_out = pd.concat([result['T'].loc[:].copy()],axis=1,keys=[ssp],names=['Scenario','Gas cycle set'])
    T_out = T_out.T.reset_index().rename({'Gas cycle set':'ensemble_member','Scenario':'scenario'},axis=1)
    T_out.loc[:,'variable'] = 'Surface Air Temperature Change'
    T_out.loc[:,'region'] = 'World'
    T_out.loc[:,'climate_model'] = 'FaIRv2.0.0-alpha'
    T_out.loc[:,'model'] = RCMIP_concs.loc[('World',ssp)].Model.unique()[0]
    T_out.loc[:,'unit'] = 'K'
    T_out = T_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()

    ## RF output
    RF_out = result['RF'].xs('carbon_dioxide',axis=1,level=-1)

    RF_out = RF_out.T.reset_index().rename({'Scenario':'ensemble_member'},axis=1)
    RF_out.loc[:,'variable'] = 'Effective Radiative Forcing|Anthropogenic|CO2'
    RF_out.loc[:,'scenario'] = ssp
    RF_out.loc[:,'region'] = 'World'
    RF_out.loc[:,'climate_model'] = 'FaIRv2.0.0-alpha'
    RF_out.loc[:,'model'] = RCMIP_concs.loc[('World',ssp)].Model.unique()[0]
    RF_out.loc[:,'unit'] = 'W/m^2'

    RF_out = RF_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()

    ## Concentrations out
    concs_out = result['C'].xs('carbon_dioxide',axis=1,level=-1)

    concs_out = concs_out.T.reset_index().rename({'Scenario':'ensemble_member'},axis=1)

    concs_out.loc[:,'variable'] = 'Atmospheric Concentrations|CO2'
    concs_out.loc[:,'scenario'] = ssp
    concs_out.loc[:,'region'] = 'World'
    concs_out.loc[:,'climate_model'] = 'FaIRv2.0.0-alpha'
    concs_out.loc[:,'model'] = RCMIP_concs.loc[('World',ssp)].Model.unique()[0]
    concs_out.loc[:,'unit'] = 'ppm'

    concs_out = concs_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()
    
    ## Emissions out
    emms_out = result['Emissions'].xs('carbon_dioxide',axis=1,level=-1)/RCMIP_outputmap.loc['carbon_dioxide','RCMIP_emms_scaling']

    emms_out = emms_out.T.reset_index().rename({'index':'ensemble_member'},axis=1)

    emms_out.loc[:,'variable'] = 'Emissions|CO2'
    emms_out.loc[:,'scenario'] = ssp
    emms_out.loc[:,'region'] = 'World'
    emms_out.loc[:,'climate_model'] = 'FaIRv2.0.0-alpha'
    emms_out.loc[:,'model'] = RCMIP_concs.loc[('World',ssp)].Model.unique()[0]
    emms_out.loc[:,'unit'] = 'Mt CO2/yr'

    emms_out = emms_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()

    output = pd.concat([T_out,RF_out,concs_out,emms_out])
    output.iloc[:,7:] = output.iloc[:,7:].astype(np.single)
    output.loc[:,'ensemble_member'] = output.loc[:,'ensemble_member'].str[3:].astype(int)

    print('saving',ssp)
    output.to_csv('../../aux/output-data/RCMIP/esm-'+ssp+'-allGHG_FaIRv2.0.0-alpha-HC5.csv.gz', compression='gzip',index=False)

In [20]:
for ssp in ['ssp119','ssp126','ssp245','ssp370','ssp370-lowNTCF-aerchemmip','ssp370-lowNTCF-gidden','ssp434','ssp460','ssp534-over','ssp585']:
    run_RCMIP_esm_ssp_allGHG(ssp)

100%|██████████| 550/550 [00:28<00:00, 19.28 timestep/s]
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:29: PerformanceWarning: indexing past lexsort depth may impact performance.
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:41: PerformanceWarning: indexing past lexsort depth may impact performance.
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:55: PerformanceWarning: indexing past lexsort depth may impact performance.
/nfs/a65/pmcjs/miniconda3/envs/leach2021/lib/python3.7/site-packages/ipykernel_launcher.py:69: PerformanceWarning: indexing past lexsort depth may impact performance.


saving ssp119


100%|██████████| 550/550 [00:25<00:00, 21.69 timestep/s]


saving ssp126


100%|██████████| 550/550 [00:29<00:00, 18.64 timestep/s]


...

saving ssp460


100%|██████████| 550/550 [00:25<00:00, 21.56 timestep/s]


saving ssp534-over


100%|██████████| 550/550 [00:25<00:00, 21.77 timestep/s]


saving ssp585
